In [1]:
def wsgi_response_middleware(start_response):
    '''
    Middleware's wrapper for `start_response`

    :param start_response: Callback
    :type start_response: callable

    :return: Wrapper
    '''
    def start_response_wrapper(status, response_headers, exc_info=None):
        print('Call start_response')
        print('Post-process response status and headers (skip)')
        if exc_info:
            print('Handle errors (skip)')
        start_response(status, response_headers, exc_info)
        print('Done start_response')
    
    return start_response_wrapper

In [2]:
def wsgi_middleware(wsgi_application):
    '''
    Middleware's wrapper for `application`

    :param wsgi_application: Application
    :type wsgi_application: callable

    :return: Wrapper
    '''
    def application_wrapper(environ, start_response):
        print('Call application')
        print('Pre-process input arguments (skip)')
        print('Wrap start_response')
        start_response = wsgi_response_middleware(start_response)
        results = wsgi_application(environ, start_response)
        if results:
            results = list(results)
            print('Post-process results (skip)')
            yield from results
        print('Done application')

    return application_wrapper

In [3]:
@wsgi_middleware
def wsgi_application(environ, start_response):
    '''Simple WSGI-application'''
    response_headers = [
        ('Content-type', 'text/plain; charset=utf-8'),
    ]
    start_response('200 OK', response_headers)
    yield b'This is wrapped!'

In [4]:
from wsgiref.simple_server import make_server

http_server = make_server('127.0.0.1', 9090, wsgi_application)
http_server.handle_request()

Call application
Pre-process input arguments (skip)
Wrap start_response
Call start_response
Post-process response status and headers (skip)
Done start_response
Post-process results (skip)
Done application


127.0.0.1 - - [19/Mar/2017 11:24:22] "GET / HTTP/1.1" 200 16
